In [ ]:
#@title Connect to Google drive { display-mode: "form" }

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# prompt: create a function that pulls a git repo, with the paramater 'repo'

import os
import subprocess
import urllib.parse


def install_pip_if_needed(package, version=None):
  """
  Installs a package if it is not already installed.

  Args:
    package: The name of the package to install.
    version: The version of the package to install (optional).
  """

  try:
    if version is not None:
      subprocess.run(["pip", "show", f"{package}=={version}"], check=True)
    else:
      subprocess.run(["pip", "show", package], check=True)
  except subprocess.CalledProcessError:
    # Package is not installed, install it
    if version is not None:
      subprocess.run(["pip", "install", f"{package}=={version}"], check=True)
    else:
      subprocess.run(["pip", "install", package], check=True)


#prompt: create function to install a package with apt,
#         with an option version number as a parameter
def install_package_apt(package_name, version=None):
  """
  Installs a package using apt.

  Args:
    package_name: The name of the package to install.
    version: The version of the package to install (optional).

  Example:
    install_package_apt("cuda-toolkit-11-8")  # Install the latest version
    install_package_apt("cuda-toolkit-11-8", "11.8.0")  # Install a specific version
  """

  if version is not None:
    command = f"apt install -y {package_name}={version}"
  else:
    command = f"apt install -y {package_name}"

  subprocess.run(command, shell=True, check=True)
  print(f"{package_name} installed successfully.")


#prompt: create function to get the last path of a url, with the paramater 'url'
def get_last_path_of_url(url):
  """
  Gets the last path of a URL.

  Args:
    url: The URL.

  Returns:
    The last path of the URL.
  """

  # Strip off any trailing slashes from the URL
  url = url.rstrip('/')
  parsed_url = urllib.parse.urlparse(url)
  print(f"Last path of {url} is {parsed_url.path.split('/')[-1]}")
  return parsed_url.path.split('/')[-1]


# prompt: create function to clone a repo if it does not already exist. if it does exist, pull the repo. parameter name is repo
def clone_or_pull_repo(repo):
  """
  Clones a git repo if it doesn't exist, otherwise pulls the repo.

  Args:
    repo: The URL of the git repo.

  Returns:
    None
  """
  newdir=get_last_path_of_url(repo)
  print(f"Repo directory is:{newdir}")
  # Check if the repo exists.
  if not os.path.exists(newdir):
    # Clone the repo.
    print(f"Cloning {repo}...")
    !git clone {repo}
  else:
    # Pull the repo.
    os.chdir(newdir)
    print(f"Pulling {repo}...")
    !git pull



#@title Setup { display-mode: "form" }
#@markdown 1. Clones repo
#@markdown 2. Cd's into repo directory
#@markdown
#@markdown Or
git_repos = "" #@param {type:"string"}
#start setup


#--upgrade numpy==1.23.5
install_package_apt('ffmpeg')
install_pip_if_needed('numpy','1.23.5')
pip_packages = [
    "gradio",
    "pydub",
    "ffmpeg",
    "ffmpeg-python",
    "torchvision",
    "torch",
    "torchaudio",
    "huggingface-hub",
    "bark",
]


# Loop through the array and install each package
for package in pip_packages:
  print(f"Installing pip {package}...")
  install_pip_if_needed(package)

os.chdir('/content')
clone_or_pull_repo('https://github.com/gitmylo/audio-webui/')
print("Done with setup!")
#end setup

In [ ]:
#@title Launching { display-mode: "form" }
username = '' #@param {type:"string"}
#@markdown Default: "`user`"<br>
#@markdown Sets the username for the webui

#@markdown

password = '' #@param {type:"string"}
#@markdown Default: "`pass`", **please change this**<br>
#@markdown Sets the password for the webui

#@markdown

args = '--theme dark' #@param {type:"string"}
#@markdown Default: empty<br>
#@markdown Sets the other args for the webui

#@markdown
os.chdir('/content/audio-webui')
args = f'-sv {args} -s -u "{username}" -p "{password}"'
!python main.py $args